In [1]:
import os
import numpy as np
import pandas as pd
import torch
import os
import json
import pandas as pd
from PIL import Image
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import cv2
import numpy as np
%matplotlib inline
from tqdm import tqdm
import glob
import pickle
import glob
import os.path as osp
DATA_DIR = '/mnt/chicm/data/open-images/relation'
DETECT_DATA_DIR = '/mnt/chicm/data/open-images/detect'
IMG_DIR = '/mnt/chicm/data/open-images/train/imgs'

In [2]:
!ls $DATA_DIR

challenge-2019-attributes-description.csv
challenge-2019-classes-vrd.csv
challenge-2019-relationships-description.csv
challenge-2019-relationship-triplets.csv
challenge-2019-train-vrd-bbox.csv
challenge-2019-train-vrd.csv
challenge-2019-train-vrd-labels.csv
challenge-2019-validation-vrd-bbox.csv
challenge-2019-validation-vrd.csv
challenge-2019-validation-vrd-labels.csv
classes_is_42.csv
negative_tuples_0804.csv
negative_tuples.csv
top_classes_42.csv
top_classes.csv
VRD_sample_submission.csv


In [3]:
df_vrd = pd.read_csv(os.path.join(DATA_DIR, 'challenge-2019-train-vrd.csv'))
df_vrd.head()

,ImageID,LabelName1,LabelName2,XMin1,XMax1,YMin1,YMax1,XMin2,XMax2,YMin2,YMax2,RelationshipLabel
0,fe58ec1b06db2bb7,/m/04bcr3,/m/083vt,0.00500,0.033125,0.580000,0.627778,0.00500,0.033125,0.580000,0.627778,is
1,82d16a22f703df5c,/m/04dr76w,/m/02gy9n,0.61200,0.735000,0.418919,0.950450,0.61200,0.735000,0.418919,0.950450,is
2,b54d41beabcfd900,/m/01mzpv,/m/083vt,0.37250,0.399375,0.706413,0.778557,0.37250,0.399375,0.706413,0.778557,is
3,4b6a08cc110d7275,/m/01mzpv,/m/01y9k5,0.17125,0.255625,0.557500,0.749167,0.20750,0.683125,0.611667,0.999167,at
4,0144cfbb726f4c72,/m/01mzpv,/m/04bcr3,0.85500,0.950000,0.561667,0.609167,0.82875,0.999375,0.568333,0.673333,at


In [4]:
df_vrd_is = df_vrd.loc[df_vrd.RelationshipLabel=='is'].copy()
df_vrd_is.head()

,ImageID,LabelName1,LabelName2,XMin1,XMax1,YMin1,YMax1,XMin2,XMax2,YMin2,YMax2,RelationshipLabel
0,fe58ec1b06db2bb7,/m/04bcr3,/m/083vt,0.005000,0.033125,0.580000,0.627778,0.005000,0.033125,0.580000,0.627778,is
1,82d16a22f703df5c,/m/04dr76w,/m/02gy9n,0.612000,0.735000,0.418919,0.950450,0.612000,0.735000,0.418919,0.950450,is
2,b54d41beabcfd900,/m/01mzpv,/m/083vt,0.372500,0.399375,0.706413,0.778557,0.372500,0.399375,0.706413,0.778557,is
8,9a87b10b780dd7e6,/m/01mzpv,/m/083vt,0.556875,0.659375,0.714822,0.826454,0.556875,0.659375,0.714822,0.826454,is
9,e4990b198db250d3,/m/01mzpv,/m/083vt,0.138750,0.199375,0.435833,0.505000,0.138750,0.199375,0.435833,0.505000,is


In [10]:
len(df_vrd_is.LabelName1.unique())

23

In [5]:
df_vrd_is.LabelName1.unique()

array(['/m/04bcr3', '/m/04dr76w', '/m/01mzpv', '/m/078n6m', '/m/0342h',
       '/m/03m3pdh', '/m/03ssj5', '/m/01_5g', '/m/01y9k5', '/m/0cvnqh',
       '/m/07y_7', '/m/071p9', '/m/05r5c', '/m/01940j', '/m/01s55n',
       '/m/080hkjn', '/m/026t6', '/m/02p5f1q', '/m/0cmx8', '/m/0dt3t',
       '/m/0584n8', '/m/04ctx', '/m/02jvh9'], dtype=object)

In [6]:
df_vrd_is.LabelName2.unique()

array(['/m/083vt', '/m/02gy9n', '/m/05z87', '/m/04lbp', '/m/0dnr7'],
      dtype=object)

# Model1:
There are 23 classes with 5 attributes, we can build a classifier model to do multi-class classification for the 23 classes, 

I tried two models 

1. CNN model
https://github.com/chicm/open-images-vrd/blob/master/vrd/train.py

2. Gradient boosting model:
https://github.com/chicm/open-images-vrd/blob/master/notebooks/submit-is-6-catboost.ipynb

Both models give me around 0.045 public lb, this is too low.

In [8]:
df_vrd_is['target'] = df_vrd_is.LabelName1.str.cat(df_vrd_is.LabelName2, sep=',')
df_vrd_is.head()

,ImageID,LabelName1,LabelName2,XMin1,XMax1,YMin1,YMax1,XMin2,XMax2,YMin2,YMax2,RelationshipLabel,target
0,fe58ec1b06db2bb7,/m/04bcr3,/m/083vt,0.005000,0.033125,0.580000,0.627778,0.005000,0.033125,0.580000,0.627778,is,"/m/04bcr3,/m/083vt"
1,82d16a22f703df5c,/m/04dr76w,/m/02gy9n,0.612000,0.735000,0.418919,0.950450,0.612000,0.735000,0.418919,0.950450,is,"/m/04dr76w,/m/02gy9n"
2,b54d41beabcfd900,/m/01mzpv,/m/083vt,0.372500,0.399375,0.706413,0.778557,0.372500,0.399375,0.706413,0.778557,is,"/m/01mzpv,/m/083vt"
8,9a87b10b780dd7e6,/m/01mzpv,/m/083vt,0.556875,0.659375,0.714822,0.826454,0.556875,0.659375,0.714822,0.826454,is,"/m/01mzpv,/m/083vt"
9,e4990b198db250d3,/m/01mzpv,/m/083vt,0.138750,0.199375,0.435833,0.505000,0.138750,0.199375,0.435833,0.505000,is,"/m/01mzpv,/m/083vt"


In [11]:
len(df_vrd_is.target.unique())

42

We can see there are totally 42 combinations of these 23 classes and 5 attributes, so we can build an object detector to detect these 42 classes.
I tried 3 detectors:

https://github.com/chicm/open-images-vrd/blob/master/configs/cas_57_800.py

https://github.com/chicm/open-images-vrd/blob/master/configs/cas_57_1024.py

https://github.com/chicm/open-images-vrd/blob/master/configs/ga_retinanet_x101_32x4d_fpn_1x_is_42.py

Each model gives me around 0.065 public lb, the ensemble model gives me 0.07 LB, this is my current best 'is' model.

toshif mentioned that he got 0.09 LB with 'is' model 
https://www.kaggle.com/c/google-ai-open-images-visual-relationship-track/discussion/64642